In [1]:
from models import *
from utils import *

import os, sys, time, datetime
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from agora_community_sdk import AgoraRTC
from io import BytesIO, StringIO
from IPython.display import clear_output
import IPython

%matplotlib notebook

In [2]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
# model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.FloatTensor

/home/samyak/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [3]:
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()[:3, ...]
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]

In [4]:
client = AgoraRTC.create_watcher('029250729b864e369e870703ac3fd265')
client.join_channel('1000')

In [5]:
users = client.get_users()
user1 = users[0]

In [ ]:
# load image and get detections
while True:
    plt.figure()
    fig, ax = plt.subplots(1, figsize=(12,9))
    
    im = user1.frame
    img = Image.open(BytesIO(im))
    detections = detect_image(img)

    img = np.array(img)
    ax.imshow(img)

    if detections is not None:
        # browse detections and draw bounding boxes
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
            bbox, color = utils.plot_helper(x1, y1, x2, y2, cls_pred, img, img_size, detections)
            ax.add_patch(bbox)
            plt.text(x1, y1, s=classes[int(cls_pred)], color='white', verticalalignment='top',
                    bbox={'color': color, 'pad': 0})
    plt.axis('off')

    f = BytesIO()
    plt.savefig(f, format="jpeg")
    plt.close()
    
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
    clear_output(wait=True)

In [7]:
client.unwatch()